In [16]:
import pandas as pd
import numpy as np

In [17]:
# have to put the urls in chronological order if you want to specify stop date. only works 1 region at a time
url_lst = ['https://docs.google.com/spreadsheets/d/13gHl6iXtmV2yqTxS9Tn2aVe-L7dacnBA5BBn0jdhcas/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1xLIST_YPe9Amhc2c9ZLYV0gOiS3wtidtRM_KhyJMu_s/edit#gid=1642263273',
'https://docs.google.com/spreadsheets/d/1N4o8nDhHVUUcylB4pHcWk9czKOLSEv-hjZcvExS3vSg/edit#gid=1642263273'
]

sheet_lst = []

counter = 0
for i in url_lst:
    if counter == 0:
        sheet_url = i
        url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
        df = pd.read_csv(url).reset_index(drop=True)
        sheet_lst.append(df)

    else:
        sheet_url = i
        url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
        df = pd.read_csv(url).reset_index(drop=True)
        sheet_lst.append(df[df.columns[5:]])
    counter += 1

if len(url_lst) > 1:
    df = pd.concat(sheet_lst, axis=1).T.reset_index().T
elif len(url_lst) == 1:
    df = df.T.reset_index().T

In [18]:
def nan_check(df, stop_date):
    """
    df: Google Sheet that has been converted to a DataFrame
    stop_date: Date to stop at as a str (M/DD/YYYY), empty string otherwise (''). Eg: If you want to stop at '4/15/2022', stop_date = '4/16/2022'
    output: dictionary indexed by state with values as a list of dates missing data
    """
    nan_okay = ['GA', 'IN', 'MI', 'CT', 'ND', 'VA', 'MA', 'ME', 'TX', 'VT', 'TN', 'PA', 'CA', 'PR', 'MN', 'WA', 'WI', 'IL', 'OH']
    filtered = df.loc[(~df[4].isin(nan_okay)) & (df[3] != np.nan) & (df[3].str.contains(',')) &
         (df[3].str.contains('District of Columbia') == False) & (df[3].str.contains('(state)') == False)].reset_index()
    first_row = pd.DataFrame(df.iloc[0]).T
    df = first_row.append(filtered, ignore_index=True)
    counties = df[3]
    state = df[4]
    missing_dict = {}
    counter_key = {0:'tstpos', 1:'pbpos', 2:'mort', 3:'pbmort'}
    counter = 0
    state_counts = df.groupby(4).count()[3]

    for i in df.loc[:,5:]:
        
        # keeps track of current date and if it matches the stop date, end the loop
        if 'Unnamed' not in str(df[i].iloc[0]):
            curr_date = str(df[i].iloc[0])
            counter = 0
            if curr_date == stop_date:
                break

        curr_col = df[i]
        # list of the indices with a NaN value
        null_idx = np.where(curr_col.isnull())[0]

        # loops over list of NaN values if it is not an empty list
        if len(null_idx > 0):
            for j in null_idx:
                nan_state = state[j]
                nan_county = counties[j]
                

                # to add counties also change dict value to (curr_date, nan_county)
                # adds the state and date to the output dictionary 
                nan_state = nan_county.split(',')[-1]
                if nan_state in missing_dict.keys():   
                    if curr_date not in missing_dict[nan_state]:
                        missing_dict[nan_state] += [curr_date]
                elif nan_state not in missing_dict.keys():
                    missing_dict[nan_state] = [curr_date]
        counter += 1

    return missing_dict
    

In [19]:
def nan_check_county(df, stop_date):
    """
    df: Google Sheet that has been converted to a DataFrame
    stop_date: Date to stop at as a str (M/DD/YYYY), empty string otherwise (''). Eg: If you want to stop at '4/15/2022', stop_date = '4/16/2022'
    output: dictionary indexed by state with values as list of tuples containing date, county, and column (mort, pbmort, tstpos, pbpos) the missing data is in
    """
    nan_okay = ['GA', 'IN', 'MI', 'CT', 'ND', 'VA', 'MA', 'ME', 'TX', 'VT', 'TN', 'PA', 'CA', 'PR', 'MN', 'WA', 'WI', 'IL', 'OH']
    filtered = df.loc[(~df[4].isin(nan_okay)) & (df[3] != np.nan) & (df[3].str.contains(',')) &
         (df[3].str.contains('District of Columbia') == False) & (df[3].str.contains('(state)') == False)].reset_index()
    first_row = pd.DataFrame(df.iloc[0]).T
    df = first_row.append(filtered, ignore_index=True)
    counties = df[3]
    state = df[4]
    missing_dict = {}
    counter_key = {0:'tstpos', 1:'pbpos', 2:'mort', 3:'pbmort'}
    counter = 0
    state_counts = df.groupby(4).count()[3]

    for i in df.loc[:,5:]:
        
        # keeps track of current date and if it matches the stop date, end the loop
        if 'Unnamed' not in str(df[i].iloc[0]):
            curr_date = str(df[i].iloc[0])
            counter = 0
            if curr_date == stop_date:
                break

        curr_col = df[i]
        # list of the indices with a NaN value
        null_idx = np.where(curr_col.isnull())[0]

        # loops over list of NaN values if it is not an empty list
        if len(null_idx > 0):
            for j in null_idx:
                nan_state = state[j]
                nan_county = counties[j]

                if nan_state in missing_dict.keys():
                    missing_dict[nan_state] += [(curr_date, nan_county, counter_key[counter])]
                elif nan_state not in missing_dict.keys():
                    missing_dict[nan_state] = [(curr_date, nan_county, counter_key[counter])]
        counter += 1


    return missing_dict
    

In [20]:
# use nan_check if you only need dates of missing data, nan_check_county if you need date, county, and column
# input date should be the day after where it should stop at
out = nan_check(df, '5/16/2022')
for i in out:
    if 'Unnamed' not in i:
        print(i,':' ,out[i])

 New Mexico : ['4/21/2022']
 Nebraska : ['5/3/2022', '5/8/2022']
